In [ ]:
import os
import sys

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch3d
from pytorch3d import _C
from pytorch3d.io import load_obj, save_obj, load_objs_as_meshes
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes, knn_points, estimate_pointcloud_normals, knn_gather, cubify
from pytorch3d.loss.point_mesh_distance import point_mesh_edge_distance, point_mesh_face_distance
import trimesh

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import pyvista as pv
pv.start_xvfb()
pv.set_jupyter_backend('html')


from torch.utils.data import DataLoader, Dataset, DistributedSampler  
from torch.nn.parallel import DistributedDataParallel as DDP  

# # from pytorch3d.structures import Meshes

# from .utils import one_hot_sparse


In [ ]:
from ops.mesh_geometry import *
# import pymeshlab as ml

In [ ]:

trimesh_tem = trimesh.load('./data_example/Bull.obj', force='mesh')

# delete a few faces to make the mesh non-watertight
# trimesh_tem.faces[590:593,:] = trimesh_tem.faces[590:591,:]
print(trimesh_tem.is_watertight)

device = torch.device("cuda:6")

mesh_tem = Meshes(verts=[torch.from_numpy(trimesh_tem.vertices).float()], 
                  faces=[torch.from_numpy(trimesh_tem.faces).long()]).to(device)

mesh_tem = normalize_mesh(mesh_tem) # normalize the mesh to fit in the unit cube

print(mesh_tem.verts_packed().shape)


In [ ]:
pl = pv.Plotter(notebook=True)

trimesh_tem = trimesh.Trimesh(vertices=mesh_tem.verts_list()[0].cpu().numpy(), faces=mesh_tem.faces_list()[0].cpu().numpy())

pl.add_mesh(trimesh_tem, color='lightblue', show_edges=True, opacity=1)


# pl.add_points(coordinates_downsampled.cpu().numpy()[np.random.choice(n_points, 1000)], color='red', point_size=5)

# pl.camera.roll = 10
pl.camera.elevation = 140
pl.camera.azimuth = 60
pl.camera.zoom = 1.3

pl.show() #screenshot='out_exp/tem_mesh_2.png', window_size=[800,800])

In [ ]:
# voxelizer = Differentiable_Voxelizer(bbox_density=128)


sample_size = 256

meshbbox = mesh_tem.get_bounding_boxes()[0]

coordinates_downsampled = torch.stack(torch.meshgrid(torch.linspace(meshbbox[0,0], meshbbox[0,1], sample_size),
                                                        torch.linspace(meshbbox[1,0], meshbbox[1,1], sample_size),
                                                        torch.linspace(meshbbox[2,0], meshbbox[2,1], sample_size)), dim=-1)

coordinates_downsampled = coordinates_downsampled.view(-1, 3).to(device)

n_points = coordinates_downsampled.shape[0]


Single GPU 

In [ ]:
with torch.no_grad():
    #sdf_result = signed_distance_field(mesh_tem, coordinates_downsampled, allow_grad=False)
    occp_result = occupancy(mesh_tem, coordinates_downsampled, allow_grad=False)


Multi-GPUs Accelerating

In [ ]:
# arctan_occp = arctan_det_occp(mesh_tem)

# # if u wanna use multi-gpus (but may not be faster)

# output_gpu = torch.device("cuda:4")

# arctan_occp = nn.DataParallel(arctan_occp, device_ids=[4,5,6,7], output_device=output_gpu)
# arctan_occp = arctan_occp.cuda()
# arctan_occp = arctan_occp.half()


# dats_set = torch.utils.data.TensorDataset(coordinates_downsampled.half().cpu(), torch.arange(0, n_points).long())

# sampler = DistributedSampler(dats_set)  

# dataloader = torch.utils.data.DataLoader(dats_set, batch_size=128**3, shuffle=False, drop_last=False, sampler=sampler)

# arctan_occp.eval()

# occp_result = torch.zeros(n_points, dtype=torch.half, device=output_gpu)

# opp_result = occp_result.half()


# for i, (data, idx) in enumerate(dataloader): ### multi-gpu
#     points = data.cuda()
#     indx = idx.to(output_gpu)
#     with torch.no_grad():
#         occp = arctan_occp.forward(points, 2000)
#         occp_result[indx] = occp


In [ ]:
dist, _ = _C.point_face_dist_forward(coordinates_downsampled.view(-1, 3).to(device),
                        torch.tensor([0], device=device, dtype=torch.int64),
                        mesh_tem.verts_packed()[mesh_tem.faces_packed(),:].to(device),
                        torch.tensor([0], device=device, dtype=torch.int64),
                        n_points, 1e-5)

occpfield = occp_result.view(1, sample_size, sample_size, sample_size)
occpfield = occpfield.permute(0, 3, 2, 1)

sdf = dist*torch.where(occp_result>0.5, -1.0, 1.0)

sdf_trg = sdf.view(1, sample_size, sample_size, sample_size)
sdf_trg = sdf_trg.permute(0, 3, 2, 1)

Re-cubify

In [ ]:

cubified = cubify(-sdf_trg, 1e-5) # cubify the voxel grid, which is the inverse operation of voxelization

cubified = cubified.update_padded(cubified.verts_padded()*(meshbbox[:,1].view(1,1, 3).to(device)))


In [ ]:
pl = pv.Plotter(notebook=True)
trimesh_cubified = trimesh.Trimesh(cubified.verts_packed().detach().cpu().numpy(), cubified.faces_packed().detach().cpu().numpy())
trimesh_cubified = trimesh.smoothing.filter_laplacian(trimesh_cubified, iterations=4)

trimesh_original = trimesh.Trimesh(mesh_tem.verts_list()[0].cpu().numpy(), mesh_tem.faces_list()[0].cpu().numpy())

# pl.add_mesh(trimesh_original, color='lightgreen', show_edges=True, opacity=0.2)

pl.add_mesh(trimesh_cubified, color='lightgreen', opacity=1, show_edges=False)

pl.camera.elevation = 140
pl.camera.azimuth = 60
pl.camera.zoom = 1.3

pl.show() 
